In [1]:
# 셀레미움 디버깅용 주피터 파일
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import os
import subprocess

# 셀레미움으로 동작하려는 코드 테스트용 주피터 파일

# 크롬이 실행되고 있으면 더 실행 하지 않으려고 만든 함수인데 동작을 안 함..
def is_chrome_running():
    try:
        output = subprocess.check_output(['tasklist'], encoding='mbcs')
        if 'chrome.exe' in output:
            for line in output.splitlines():
                if 'chrome.exe' in line and '--remote-debugging-port=9222' in line:
                    return True
        return False
    except subprocess.CalledProcessError:
        return False

# 설치된 크롬을 옵션 주어서 실행하는 함수, 유저 프로파일도 분리
# 설치된 경로를 자신의 컴퓨터 경로로 수정필요
# 처음 User Data로 필요한 웹페이지 로그인 후 해당 유저 데이터를 복사해서 이름 변경
def start_chrome(option):
    user_data_dirs = {
        1: (r"C:\Users\jaehee\Downloads\automatic\User Data", 9222),
        2: (r"C:\Users\jaehee\Downloads\automatic\WhatsApp Data", 9223),
        3: (r"C:\Users\jaehee\Downloads\automatic\Slack Data", 9224)
    }
    if option not in user_data_dirs:
        user_data_dir = r"C:\Users\jaehee\Downloads\automatic\User Data"

    chrome_path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
    user_data_dir, port = user_data_dirs[option]

    if is_chrome_running():
        print("Chrome이 이미 디버깅 모드로 실행중입니다.")
        return 

    subprocess.Popen([chrome_path, '--remote-debugging-port={}'.format(port), f'--user-data-dir={user_data_dir}'])
    time.sleep(1)

    return port

# 크롬 드라이버로 실행할 수 있는데 유저 데이터값이 유지가 안 되고 같은 세션으로 접속시키기 어려움
# chrome://settings/help 에서 크롬 버전 확인                  ↓여기에 크롬 버전 입력해서 다운로드
# https://storage.googleapis.com/chrome-for-testing-public/128.0.6613.119/win64/chromedriver-win64.zip 
def setup_driver(port):
    os.environ['WDM_SSL_VERIFY'] = '0'  # SSL 인증서 검증 비활성화
    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    # chrome_options.add_argument("--start-fullscreen")  # 전체화면 모드로 실행
    chrome_options.add_experimental_option("debuggerAddress", f"127.0.0.1:{port}")  # 포트 설정

    chrome_driver_path = r"C:\Users\jaehee\Downloads\automatic\chromedriver-win64\chromedriver.exe" # 여기에 다운로드한 크롬드라이버 위치 입력
    service = Service(executable_path=chrome_driver_path)

    # service = Service(ChromeDriverManager().install()) # 이게 크롬 드라이버 자동 설치인데 자꾸 버전이 안맞음

    return webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# 기본으로 백그라운드에서 동작하기에, ui모드로 동작하는 인자 추가
# 이후 필요한 페이지 로그인 하면 됨
# 그리고 나서 유저 데이터 2개를 더 복사해서 생성
port = start_chrome(1, headless=False)
driver = setup_driver(port, headless=False)
driver.get("google.com")